In [1]:
import pandas as pd
import os

# romanian months to month_number format
romanian_months = {
    'ianuarie': 1, 'februarie': 2, 'martie': 3, 'aprilie': 4,
    'mai': 5, 'iunie': 6, 'iulie': 7, 'august': 8,
    'septembrie': 9, 'octombrie': 10, 'noiembrie': 11, 'decembrie': 12
}

# translation into English of column names
english_columns = {
    'Județ': 'county',
    'Număr terenuri': 'plots',
    'Nr. Unități Individuale': 'individual_units',
    'Extravilan': 'rural',
    'Intravilan': 'urban',
    'Agricol': 'agricultural',
    'Neagricol': 'non_agricultural',
    'Cu construcții': 'with_construction',
    'Fără construcții': 'without_construction'
}

# get the date from the folder name, to add a 'date' column to the cleaned data frame
def extract_date_from_folder(folder_name):
    parts = folder_name.lower().split('-')
    if len(parts) == 3:
        ro_month = parts[1]
        year = parts[2]
        month_number = romanian_months.get(ro_month)
        if month_number:
            return pd.to_datetime(f"{year}-{month_number}-1")
    return None

# define MultiIndex header. Skipped first row (title) and last row (total)
def clean_excel_file(filepath, date):
    df = pd.read_excel(filepath, header=[1, 2, 3], skiprows=1, skipfooter=1)

    # drop 'Total' columns
    columns_to_drop = [col for col in df.columns if 'Total' in str(col[0])]
    df = df.drop(columns=columns_to_drop)

    # drop first column (index)
    df = df.drop(df.columns[0], axis=1)

    # replace 'unnamed' with an empty string
    df.columns = pd.MultiIndex.from_tuples(
        tuple('' if 'Unnamed' in str(x) else x for x in tup)
        for tup in df.columns
    )

    # translate columns from romanian to english
    df.columns = pd.MultiIndex.from_tuples(
        tuple(english_columns.get(str(x), x) for x in tup)
        for tup in df.columns
    )

    # melt the data frame
    id_vars = [col for col in df.columns if col[0].lower() == 'county']
    df_melted = df.melt(id_vars=id_vars)
    df_melted.columns = ['county', 'property_type', 'location_type', 'usage_type', 'quantity']

    # insert a new column with the date
    df_melted.insert(0, 'date', date)

    return df_melted

# function to loop through all the folders and files 

def process_all_data(data_path='data', output_dir='cleaned_tables'):
    os.makedirs(output_dir, exist_ok=True)
    all_data = []


    for folder_name in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder_name)

        if os.path.isdir(folder_path):
            # Get the date (YYYY-MM-01) from the folder name to add the column 'date'
            date = extract_date_from_folder(folder_name)
            if date is None:
                continue
            
            # get the file name (files containing 'vanzari')
            for filename in os.listdir(folder_path):
                if filename.endswith('.xlsx') and 'vanzari' in filename.lower():
                    file_path = os.path.join(folder_path, filename)
                    try:
                        # Clean the Excel file and add the folder date to each row
                        cleaned_df = clean_excel_file(file_path, date)
                        all_data.append(cleaned_df)
                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        output_file = os.path.join(output_dir, 'selling_table.csv')
        final_df.to_csv(output_file, index=False)
        print(f"Combined CSV saved to: {output_file}")
    else:
        print("⚠️ No valid files were processed.")

# Run
process_all_data()


Error processing data/statistica-aprilie-2024/2024-ian_aprilie_vanzari.xlsx: 'int' object has no attribute 'lower'
Combined CSV saved to: cleaned_tables/selling_table.csv


In [2]:
import pandas as pd

df = pd.read_csv('cleaned_tables/selling_table.csv')
display(df.head())
display(df.columns.to_list())

,date,county,property_type,location_type,usage_type,quantity
0,2022-06-01,ALBA,plots,rural,agricultural,149.0
1,2022-06-01,ARAD,plots,rural,agricultural,355.0
2,2022-06-01,ARGEȘ,plots,rural,agricultural,158.0
3,2022-06-01,BACAU,plots,rural,agricultural,136.0
4,2022-06-01,BIHOR,plots,rural,agricultural,317.0


['date', 'county', 'property_type', 'location_type', 'usage_type', 'quantity']